# Tracking Experiments

With ML projects, we usually train 20+ models, and which introduces a different set of problems. Tracking of these experiments is important and there are various tools just for this

1. In built pytorch's board
2. 3rd party tools like weights and bias https://wandb.ai/site/
3. mlflow - another 3rd party

there are many more solutsion

In [3]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

In [ ]:
# from going_modular.going_modular import data_setup, engine
if not IN_COLAB:
    try:
        from going_modular.going_modular import data_setup, engine
    except ImportError:
        import os
        import sys
        from pathlib import Path
        sys.path.append(str(Path(os.path.abspath('')).parent))
        from going_modular.going_modular import data_setup
else: 
    try:
        from going_modular.going_modular import data_setup, engine
    except:
        # Get the going_modular scripts
        print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
        !git clone https://github.com/harshankbansal/pytorch-deep-learning
        !mv pytorch-deep-learning/going_modular .
        !mv helper_functions.py .
        !mv data .
        !rm -rf pytorch-deep-learning
        from going_modular.going_modular import data_setup, engine

import helper_functions